In [1]:
import numpy as np
import pandas as pd
import os
import datetime
import math
import tensorflow as tf
import matplotlib.pyplot as plt

from tcn import TCN
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, Concatenate
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from keras.optimizers import Adam


2024-06-14 12:34:02.214931: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-14 12:34:02.234516: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-14 12:34:02.234532: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-14 12:34:02.234544: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-14 12:34:02.238603: I tensorflow/core/platform/cpu_feature_g

In [2]:
print(tf.__version__)
print(tf.config.list_physical_devices())
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

2.14.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
2 Physical GPUs, 1 Logical GPU


2024-06-14 12:34:03.303263: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-14 12:34:03.303393: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-14 12:34:03.307131: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
SEED = 42
tf.keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

In [4]:
# dirs
DATA_DIR = "./load.csv"
TEST_PLOT_DIR = "./test_plots/TCN_LSTM/"
load_col = 'out.site_energy.total.energy_consumption.kwh'

In [5]:
if not os.path.exists(TEST_PLOT_DIR):
    os.makedirs(TEST_PLOT_DIR)
if not os.path.exists("./model"):
    os.makedirs("./model")
if not os.path.exists("./training_history"):
    os.makedirs("./training_history")

In [6]:
data = pd.read_csv(DATA_DIR)
data['timestamp'] = pd.to_datetime(data['timestamp'])
data[load_col] = data[load_col] * 4 / 1e3


In [7]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[load_col].to_numpy().reshape(-1, 1))
data[load_col] = data_scaled

In [8]:
data.describe()

,upgrade,timestamp,models_used,floor_area_represented,out.district_cooling.cooling.energy_consumption.kwh,out.district_heating.heating.energy_consumption.kwh,out.district_heating.water_systems.energy_consumption.kwh,out.electricity.cooling.energy_consumption.kwh,out.electricity.exterior_lighting.energy_consumption.kwh,out.electricity.fans.energy_consumption.kwh,...,out.electricity.total.energy_consumption.kwh.savings,out.natural_gas.total.energy_consumption.kwh.savings,out.district_heating.cooling.energy_consumption.kwh.savings,out.natural_gas.cooling.energy_consumption.kwh.savings,out.other_fuel.cooling.energy_consumption.kwh.savings,out.other_fuel.heating.energy_consumption.kwh.savings,out.other_fuel.interior_equipment.energy_consumption.kwh.savings,out.other_fuel.total.energy_consumption.kwh.savings,out.other_fuel.water_systems.energy_consumption.kwh.savings,out.site_energy.total.energy_consumption.kwh.savings
count,35040.0,35040,35040.0,3.504000e+04,35040.000000,35040.0,35040.0,35040.000000,35040.000000,35040.000000,...,35040.000000,35040.000000,35040.0,35040.0,35040.0,35040.0,35040.0,35040.0,35040.0,35040.000000
mean,18.0,2018-07-02 12:07:30,535.0,2.933235e+08,351.077590,0.0,0.0,10702.802560,3606.591142,20721.475338,...,4979.811962,23848.451624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28859.420341
min,18.0,2018-01-01 00:15:00,535.0,2.933235e+08,0.000000,0.0,0.0,326.378723,0.000000,16646.463233,...,-28508.867274,262.091277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3108.391514
25%,18.0,2018-04-02 06:11:15,535.0,2.933235e+08,56.488275,0.0,0.0,2706.386049,0.000000,19725.608356,...,-529.918116,8719.805585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17223.696432
50%,18.0,2018-07-02 12:07:30,535.0,2.933235e+08,259.402965,0.0,0.0,7965.151518,3546.633502,20458.531255,...,2273.919309,18191.504378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23244.709423
75%,18.0,2018-10-01 18:03:45,535.0,2.933235e+08,574.228447,0.0,0.0,15479.805245,7127.583771,21985.597167,...,9286.504567,32494.864830,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35158.530405
max,18.0,2019-01-01 00:00:00,535.0,2.933235e+08,1559.617664,0.0,0.0,55401.017935,7361.098134,24359.717506,...,40740.822629,159204.018407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,136399.915739
std,0.0,NaN,0.0,5.960550e-08,331.710202,0.0,0.0,9898.426784,3564.157927,1548.734876,...,8136.228348,20711.247693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17025.161426


In [9]:
# Define a function to generate a list of timestamps every 2 hours within the dataset's range
def generate_timestamps(data) -> pd.DatetimeIndex:
    start = data['timestamp'].min() + DateOffset(days=3)
    end = data['timestamp'].max() - DateOffset(minutes=96*15)
    timestamps = pd.date_range(start=start, end=end, freq='15min')
    return timestamps

In [10]:
timestamps = generate_timestamps(data)
print(timestamps)

DatetimeIndex(['2018-01-04 00:15:00', '2018-01-04 00:30:00',
               '2018-01-04 00:45:00', '2018-01-04 01:00:00',
               '2018-01-04 01:15:00', '2018-01-04 01:30:00',
               '2018-01-04 01:45:00', '2018-01-04 02:00:00',
               '2018-01-04 02:15:00', '2018-01-04 02:30:00',
               ...
               '2018-12-30 21:45:00', '2018-12-30 22:00:00',
               '2018-12-30 22:15:00', '2018-12-30 22:30:00',
               '2018-12-30 22:45:00', '2018-12-30 23:00:00',
               '2018-12-30 23:15:00', '2018-12-30 23:30:00',
               '2018-12-30 23:45:00', '2018-12-31 00:00:00'],
              dtype='datetime64[ns]', length=34656, freq='15min')


In [11]:
def generate_sets_for_all_timestamps(timestamps, data):
    training_sets = []
    change_rate_sets = []
    target_sets = []
    training_sets_time = []
    target_sets_time = []

    for timestamp in timestamps:
        # Calculate the range for the current period's data
        start_time_current = timestamp - DateOffset(days=2, hours=23, minutes=45)
        end_time_current = timestamp

        # Calculate the target range (the next 10 steps after the current timestamp)
        target_start_time = timestamp + DateOffset(minutes=15)
        target_end_time = timestamp + DateOffset(hours=3, minutes=0) 

        # Filter the data for training and target sets
        current_data = data[(data['timestamp'] >= start_time_current) & (data['timestamp'] <= end_time_current)]
        target_data = data[(data['timestamp'] >= target_start_time) & (data['timestamp'] <= target_end_time)]

        # Combine current and last week data for the training set
        training_data = pd.concat([current_data]).reset_index(drop=True)
        
        # Save the training and target sets
        if not training_data.empty and not target_data.empty:
            training_sets.append(training_data[load_col])
            target_sets.append(target_data[load_col])
            training_sets_time.append(list(training_data['timestamp']))
            target_sets_time.append(list(target_data['timestamp']))

    training_sets = np.array(training_sets)
    target_sets = np.array(target_sets)
    training_sets_time = np.array(training_sets_time)
    target_sets_time = np.array(target_sets_time)

    return training_sets, target_sets, training_sets_time, target_sets_time


In [12]:
# Generate training and target sets for all the timestamps
training_sets, target_sets, training_sets_time, target_sets_time = generate_sets_for_all_timestamps(timestamps, data)

In [13]:
MONTH_TIME_STEP = math.floor(timestamps.shape[0] / 24)
X_test = []
y_test = []
X_test_time = []
y_test_time = []
minList = []
maxList = []
for i in range(0, 24):
    min = (i+1)*MONTH_TIME_STEP-(192*(i+1))
    max = (i+1)*MONTH_TIME_STEP-(192*i)
    X_test.append(training_sets[min:max])
    y_test.append(target_sets[min:max])
    X_test_time.append(training_sets_time[min:max])
    y_test_time.append(target_sets_time[min:max])
    training_sets = np.concatenate([training_sets[:min], training_sets[max:]])
    target_sets = np.concatenate([target_sets[:min], target_sets[max:]])
    training_sets_time = np.concatenate([training_sets_time[:min], training_sets_time[max:]])
    target_sets_time = np.concatenate([target_sets_time[:min], target_sets_time[max:]])


In [14]:
X_test = np.concatenate([i for i in X_test])
y_test = np.concatenate([i for i in y_test])
X_test_time = np.concatenate([i for i in X_test_time])
y_test_time = np.concatenate([i for i in y_test_time])

In [15]:
X_train = training_sets.reshape(*training_sets.shape, 1)
X_test = X_test.reshape(*X_test.shape, 1)
y_train = target_sets
X_train_time = training_sets_time
y_train_time = target_sets_time

In [16]:
print(np.array(X_train).shape)
print(np.array(X_test).shape)
print(np.array(y_train).shape)
print(np.array(y_test).shape)


(30048, 288, 1)
(4608, 288, 1)
(30048, 12)
(4608, 12)


In [17]:
def create_model(input_shape, num_outputs):
    inputs = Input(shape=input_shape)

    # TCN layer
    x = TCN(activation="tanh", dilations=(1, 2, 4, 8, 16), padding='causal', return_sequences=True)(inputs)
    x = TCN(activation="tanh", dilations=(1, 2, 4, 8, 16), padding='causal', return_sequences=True)(x)
    x = TCN(activation="tanh", dilations=(1, 2, 4, 8, 16), padding='causal', return_sequences=False)(x)

    # BiLSTM layer for understanding the temporal dependencies
    x = LSTM(50, return_sequences=True)(x)
    x = LSTM(100, return_sequences=True)(x)
    x = LSTM(150, return_sequences=False)(x)


    # A Dense layer for output
    x = Dense(96, activation="linear")(x)
    x = Dense(48, activation="linear")(x)

    outputs = Dense(num_outputs, activation='linear')(x)  # For regression output

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [18]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

%load_ext tensorboard

tensorboard_callback = TensorBoard(logdir, histogram_freq=1)
early_stopping_callback = EarlyStopping(monitor="loss", patience=10, min_delta=5e-5)
reduce_lr_callback = ReduceLROnPlateau(monitor="loss", factor=0.3, patience=5, verbose=1, min_lr=1e-7)
callbacks=[tensorboard_callback, early_stopping_callback, reduce_lr_callback]
model = create_model(input_shape=X_train.shape[1:], num_outputs=12)
model.compile(optimizer=Adam(learning_rate=5e-5), loss="mse")
model.summary()

ValueError: Input 0 of layer "lstm" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 64)

: 

In [ ]:
history = model.fit(
    X_train,
    y_train,
    verbose=1,
    epochs=120,
    batch_size=96,
    callbacks=[tensorboard_callback, early_stopping_callback, reduce_lr_callback]
)

Epoch 1/120


2024-06-14 12:23:36.528244: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-06-14 12:23:36.843678: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fbe1ee43f90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-14 12:23:36.843694: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1080 Ti, Compute Capability 6.1
2024-06-14 12:23:36.846131: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-14 12:23:36.893999: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


313/313 [==============================] - 32s 82ms/step - loss: 0.1815 - lr: 5.0000e-05
Epoch 2/120
313/313 [==============================] - 26s 82ms/step - loss: 0.1120 - lr: 5.0000e-05
Epoch 3/120
313/313 [==============================] - 26s 83ms/step - loss: 0.1306 - lr: 5.0000e-05
Epoch 4/120
313/313 [==============================] - 26s 83ms/step - loss: 0.1164 - lr: 5.0000e-05
Epoch 5/120
313/313 [==============================] - 26s 83ms/step - loss: 0.1028 - lr: 5.0000e-05
Epoch 6/120
313/313 [==============================] - 26s 83ms/step - loss: 0.0831 - lr: 5.0000e-05
Epoch 7/120
313/313 [==============================] - 26s 83ms/step - loss: 0.0811 - lr: 5.0000e-05
Epoch 8/120
313/313 [==============================] - 26s 83ms/step - loss: 0.0764 - lr: 5.0000e-05
Epoch 9/120
313/313 [==============================] - 26s 83ms/step - loss: 0.0622 - lr: 5.0000e-05
Epoch 10/120
313/313 [==============================] - 26s 83ms/step - loss: 0.0578 - lr: 5.0000e-05
E

KeyboardInterrupt: 

In [ ]:
y_pred = model.predict([X_test])

In [ ]:
print(y_test.shape)
print(y_pred.shape)

mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("-" * 86)
print(f'mse: {mse:.4f}')
print(f'rmse: {rmse:.4f}')
print(f'mae: {mae:.4f}')
print(f'mape: {mape: .4f}')
print(f'r2: {r2:.4f}')
print("-" * 86)

y_test_inv = scaler.inverse_transform(y_test)
y_pred_inv = scaler.inverse_transform(y_pred)


mse_inv = mean_squared_error(y_test_inv, y_pred_inv)
rmse_inv = math.sqrt(mse_inv)
mae_inv = mean_absolute_error(y_test_inv, y_pred_inv)
mape_inv = mean_absolute_percentage_error(y_test_inv, y_pred_inv)
r2_inv = r2_score(y_test_inv, y_pred_inv)

print("-" * 86)
print(f'mse_inv: {mse_inv:.4f}')
print(f'rmse_inv: {rmse_inv:.4f}')
print(f'mae_inv: {mae_inv:.4f}')
print(f'mape_inv: {mape_inv: .4f}')
print(f'r2_inv: {r2_inv:.4f}')
print("-" * 86)

In [ ]:
def calculate_metrics_per_steps(true_values, predicted_values):
    n_steps = true_values.shape[1]

    mse = []
    rmse = []
    mae = []
    mape = []

    for i in range(n_steps):
        true_step = true_values[:, i]
        predicted_step = predicted_values[:, i]

        mse_step = mean_squared_error(true_step, predicted_step)
        rmse_step = np.sqrt(mse_step)
        mae_step = mean_absolute_error(true_step, predicted_step)
        mape_step = mean_absolute_percentage_error(true_step, predicted_step)

        mse.append(mse_step)
        rmse.append(rmse_step)
        mae.append(mae_step)
        mape.append(mape_step)

    return np.array(mse), np.array(rmse), np.array(mae), np.array(mape)

In [ ]:
model.save("./model/TCN_LSTM_12steps.keras")